## Doctor appointment booking System

This chat agent can interact with users and perform the following tasks:

    1. Select a doctor and choose a time slot for booking

    2. Cancel a booked appointment

    3. Reschedule a booked appointment based on availability

    4. Search for doctor details filtered by name, availability, specialty, and location

### 1. Connect with Gemini API

connect to the llm using google api key.

In [3]:
# connect with gemini api
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
# initialize llm chat model
from langchain_google_genai import ChatGoogleGenerativeAI
from google.genai.types import GenerateContentConfig


system_instruction = """
                        You are a phone-agent assistant.  
                        
                        Keep all messages very short—just a single question or one answer.  
                        
                        Act as if you’re on a live call: no long explanations or chit-chat.  
                        
                        Always reference prior chat context and stay aligned with the conversation.  
                        
                    """

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
    # other params...
)

### 2. State and Assistant class

This class is used to define specialized assistants based on each tasks

In [4]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages

# state of the graph
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_name: str # name of the current user
    user_id: str # id of the current user


class Assistant:
    # this initialize a runnable that can be prompt template | llm with tools
    # by doing that we can make any assistant with same code
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        # if the llm not produce a result
        # it safely update the state with in a node using state = {**state, "messages": messages}
        # normally state is updated after the node is executed 
        # in this case 
        while True:
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}

### 3. Completion of task and hand over to other assistant

This tool is called when the task is completed or the user intend to go another task

In [5]:
from pydantic import BaseModel, Field

class CompleteOrEscalate(BaseModel):
    """A tool to mark the current task as completed and/or to escalate control of the dialog to the main assistant,
    who can re-route the dialog based on the user's needs."""

    cancel: bool = True
    reason: str

    class Config:
        json_schema_extra = {
            "example": {
                "cancel": True,
                "reason": "User changed their mind about the current task.",
            },
            "example 2": {
                "cancel": True,
                "reason": "I have fully completed the task.",
            },
            "example 3": {
                "cancel": False,
                "reason": "I need to search the user's emails or calendar for more information.",
            },
        }

### 4. Create DataBase and insert sample data to that

This make a database and add some sample data 

In [12]:
import sqlite3
from datetime import datetime

# Step 1: Connect
DB_PATH = "hospital3.db"
conn = sqlite3.connect(DB_PATH) 
cursor = conn.cursor()

# Step 2: Create tables
cursor.execute("""
CREATE TABLE Doctor (
    Doctor_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Doctor_Name TEXT,
    Specialization TEXT,
    Location TEXT,
    Rating REAL
)
""")

cursor.execute("""
CREATE TABLE Patient (
    Patient_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Patient_Name TEXT,
    Age INTEGER
)
""")

cursor.execute("""
CREATE TABLE Appointment (
    Appointment_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Appointment_Time TEXT,
    Appointment_Date TEXT,
    Doctor_ID INTEGER,
    Patient_ID INTEGER,
    FOREIGN KEY (Doctor_ID) REFERENCES Doctor(Doctor_ID),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID)
)
""")

# Insert data into Doctor and Patient tables
doctors = [
    ("Dr. Alice Smith", "Cardiology", "New York", 4.7),
    ("Dr. Bob Johnson", "Neurology", "Los Angeles", 4.5),
    ("Dr. Clara Lee", "Pediatrics", "Chicago", 4.8)
]

patients = [
    ("John Doe", 30),
    ("Emily Davis", 25),
    ("Michael Brown", 45)
]

cursor.executemany("INSERT INTO Doctor (Doctor_Name, Specialization, Location, Rating) VALUES (?, ?, ?, ?)", doctors)
cursor.executemany("INSERT INTO Patient (Patient_Name, Age) VALUES (?, ?)", patients)

# Retrieve IDs
cursor.execute("SELECT Doctor_ID FROM Doctor ORDER BY Doctor_ID")
doctor_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT Patient_ID FROM Patient ORDER BY Patient_ID")
patient_ids = [row[0] for row in cursor.fetchall()]

# Insert into Appointment table
appointments = [
    ("10:00", "2025-06-06", doctor_ids[0], patient_ids[0]),
    ("11:30", "2025-06-06", doctor_ids[1], patient_ids[1]),
    ("14:00", "2025-06-07", doctor_ids[2], patient_ids[2])
]

cursor.executemany("""
    INSERT INTO Appointment (Appointment_Time, Appointment_Date, Doctor_ID, Patient_ID)
    VALUES (?, ?, ?, ?)
""", appointments)

# Step 4: Commit and verify
conn.commit()

# Print the data
for row in cursor.execute("SELECT * FROM Appointment"):
    print(row)

# Close connection
conn.close()

(1, '10:00', '2025-06-06', 1, 1)
(2, '11:30', '2025-06-06', 2, 2)
(3, '14:00', '2025-06-07', 3, 3)


### 5. Tools for searching a Doctor and book an Appointment

In [13]:
from langchain_core.tools import tool
from typing import Optional, Dict, List
import sqlite3

# Tool to search for a doctor by name
@tool
def search_for_doctor(name: Optional[str] = None) -> List[Dict]:
    """
    Search for doctors by name (or return all if name is None).

    Args:
        name (Optional[str]): The partial or full name of the doctor.

    Returns:
        List[Dict]: Doctors matching the search criteria.
    """
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    if name:
        cursor.execute("""
            SELECT * FROM Doctor WHERE Doctor_Name LIKE ?
        """, (f"%{name}%",))
    else:
        cursor.execute("SELECT * FROM Doctor")

    results = [dict(row) for row in cursor.fetchall()]
    conn.close()
    return results

# Tool to check a doctor's availability at a given date and time
@tool
def check_doctor_availability(doctor_id: int, date: str, time: str) -> Dict:
    """
    Check if a doctor is available at a given date and time.

    Args:
        doctor_id (int): The ID of the doctor.
        date (str): Date of the appointment in 'YYYY-MM-DD' format.
        time (str): Time of the appointment (e.g., '10:00').

    Returns:
        Dict: {"available": True/False}
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT 1 FROM Appointment
        WHERE Doctor_ID = ? AND Appointment_Date = ? AND Appointment_Time = ?
    """, (doctor_id, date, time))

    result = cursor.fetchone()
    conn.close()

    return {"available": result is None}


In [14]:
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime

book_doctor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized assistant for handling doctor appointment bookings. "
            "The primary assistant delegates work to you whenever the user needs help finding a doctor or scheduling an appointment. "
            "Search for doctors based on the user's preferences (name, specialization, location), and check their availability for a given date and time. "
            "Confirm the details with the user before attempting to book the appointment. "
            "If the doctor is unavailable, offer alternate time slots or suggest a similar doctor. "
            "Always confirm availability before booking. "
            "Do not proceed with a booking unless the relevant tool has successfully returned availability."
            "\nCurrent time: {time}."
            '\n\nIf the user needs help outside of your scope, then "CompleteOrEscalate" the dialog to the main assistant. '
            "Do not make up tools or waste the user's time."
            "\n\nSome examples where you should CompleteOrEscalate:\n"
            " - 'I'm looking for medical insurance info'\n"
            " - 'I want to book a hospital room'\n"
            " - 'Actually, I’ll call the clinic myself'\n"
            " - 'I want to book a flight to the hospital location first'\n"
            " - 'Doctor appointment already confirmed'\n",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now())

book_appointment_tools = [check_doctor_availability, search_for_doctor]
book_appointment_runnable = book_doctor_prompt | llm.bind_tools(
    book_appointment_tools + [CompleteOrEscalate]
)

### 6. Make the book appointment sub graph

This 

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.prebuilt import tools_condition